In [0]:
import pickle
import boto3
import datetime
import re
import json
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)
import numpy as np
import unicodedata
import matplotlib.pyplot as plt
from unidecode import unidecode
import random
import heroku3
import psycopg2

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField,LongType,TimestampType

In [0]:
from changing_affs import matching_based_on_current_affs
from institution_string_matching import string_matching_function

# create udfs
matching_based_on_current_affs_udf = F.udf(matching_based_on_current_affs, ArrayType(LongType()))
string_matching_function_udf = F.udf(string_matching_function, ArrayType(LongType()))

### Getting all data

In [0]:
def get_secret(secret_name = "prod/psqldb/conn_string"):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    
    get_secret_value_response = client.get_secret_value(
            SecretId=secret_name)

    # Decrypts secret using the associated KMS key.
    secret_string = get_secret_value_response['SecretString']
    
    secret = json.loads(secret_string)
    return secret

def shutdown_dynos(heroku_secret_token, dynos_to_stop):
    heroku_conn = heroku3.from_key(heroku_secret_token)
    app = heroku_conn.app("openalex-guts")
    current_quantities = []

    for dyno_name in dynos_to_stop:
        current_quantities.append(app.process_formation()[dyno_name].quantity)
        app.process_formation()[dyno_name].scale(0)
    return current_quantities

def restart_dynos(heroku_secret_token, dynos_to_stop, old_quantities):
    heroku_conn = heroku3.from_key(heroku_secret_token)
    app = heroku_conn.app("openalex-guts")

    for dyno_name, dyno_quantity in zip(dynos_to_stop, old_quantities):
        app.process_formation()[dyno_name].scale(dyno_quantity)

def connect_to_db():
    secret = get_secret()
    conn = psycopg2.connect( 
        host=secret['host'],
        port=secret['port'],
        user=secret['username'],
        password=secret['password'],
        database=secret['dbname']
    )
    return conn

In [0]:
secret = get_secret()
buckets = get_secret("prod/aws/buckets")
heroku_secret = get_secret(secret_name = "prod/heroku/oauth")

In [0]:
database_copy_save_path = f"{buckets['database_copy_save_path']}"
start_datetime = datetime.datetime.now()
curr_date = start_datetime.strftime("%Y_%m_%d_%H_%M")
# curr_date = '2024_07_14_23_18'
iteration_save_path = f"{buckets['temp_save_path']}/institution_fixes_{curr_date}/"

In [0]:
institutions = spark.read.parquet(f"{database_copy_save_path}/mid/institution")\
    .filter(F.col('merge_into_id').isNull()).filter(F.col('created_date').startswith('2024-')) \
    .select('affiliation_id','ror_id','display_name')

In [0]:
spark.read.parquet(f"{database_copy_save_path}/mid/institution").filter(F.col('merge_into_id').isNull()).select('affiliation_id','ror_id','display_name') \
    .toPandas().to_parquet("institutions_current.parquet")

In [0]:
affiliations = spark.read.parquet(f"{database_copy_save_path}/mid/affiliation") \
    .select('paper_id','affiliation_id').dropDuplicates() \
    .join(institutions, how='inner', on='affiliation_id')

#### Functions

In [0]:
@udf(returnType=ArrayType(LongType()))
def join_aff_id_cols(aff_ids, over_aff_ids):
    if isinstance(aff_ids, list):
        if isinstance(over_aff_ids, list):
            if (-1 in over_aff_ids) & (len(over_aff_ids) > 1):
                return [x for x in over_aff_ids if x != -1].copy()
            elif -1 in over_aff_ids:
                return over_aff_ids.copy()
            elif (len(over_aff_ids) > 0) & (isinstance(over_aff_ids[0], int)):
                return over_aff_ids.copy()
            else:
                return aff_ids
        else:
            return aff_ids
    else:
        return [-1]

#### Downloading table for fixing

In [0]:
testing = # read from redshift

In [0]:
testing.count()

140503605

In [0]:
testing \
    .write.mode('overwrite') \
    .parquet(f"{database_copy_save_path}/mid/affiliation_string_v2")

In [0]:
aff_string_table = spark.read.parquet(f"{database_copy_save_path}/mid/affiliation_string_v2") \
        .na.fill('[-1]', subset=['affiliation_ids']) \
    .na.fill('[]', subset=['affiliation_ids_override']) \
    .withColumn('new_affiliation_ids', F.regexp_replace(F.col("affiliation_ids"), " ", "")) \
    .withColumn('new_affiliation_ids', F.split(F.split(F.col('new_affiliation_ids'), '\[').getItem(1), '\]').getItem(0)) \
    .withColumn('new_affiliation_ids', F.split(F.col('new_affiliation_ids'), ',')) \
    .withColumn('new_affiliation_ids_override', 
                F.regexp_replace(F.col("affiliation_ids_override"), " ", "")) \
    .withColumn('new_affiliation_ids_override', 
                F.split(F.split(F.col('new_affiliation_ids_override'), '\[').getItem(1), '\]').getItem(0)) \
    .withColumn('new_affiliation_ids_override', F.split(F.col('new_affiliation_ids_override'), ',')) \
    .select('original_affiliation',F.col('new_affiliation_ids').cast(ArrayType(LongType())).alias('affiliation_ids'),
            F.col('new_affiliation_ids_override').cast(ArrayType(LongType())).alias('affiliation_ids_override')) \
    .dropDuplicates(subset=['original_affiliation']) \
    .withColumn('current_affs', join_aff_id_cols(F.col('affiliation_ids'), 
                                                             F.col('affiliation_ids_override')))
    
aff_string_table.cache().count()

<>:5: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\['
<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:5: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\['
<>:10: SyntaxWarning: invalid escape sequence '\]'
/root/.ipykernel/3869/command-3978419140262217-1389652454:5: SyntaxWarning: invalid escape sequence '\['
  .withColumn('new_affiliation_ids', F.split(F.split(F.col('new_affiliation_ids'), '\[').getItem(1), '\]').getItem(0)) \
/root/.ipykernel/3869/command-3978419140262217-1389652454:5: SyntaxWarning: invalid escape sequence '\]'
  .withColumn('new_affiliation_ids', F.split(F.split(F.col('new_affiliation_ids'), '\[').getItem(1), '\]').getItem(0)) \
/root/.ipykernel/3869/command-3978419140262217-1389652454:10: SyntaxWarning: invalid escape sequence '\['
  F.split(F.split(F.col('new_affiliati

140130473

#### Running the string matching functions

In [0]:
@udf(returnType=IntegerType())
def check_for_change(aff_ids, over_aff_ids):
    curr_set = set(aff_ids)
    new_set = set(over_aff_ids)
    if curr_set == new_set:
        return 0
    elif (len(curr_set) == 1) and (len(new_set) == 1) and (not isinstance(list(curr_set)[0], int)) and (list(over_aff_ids)[0]==-1):
        return 0
    else:
        return 1

In [0]:
@udf(returnType=ArrayType(LongType()))
def remove_negative_1(aff_ids):
    new_aff_ids = [x for x in aff_ids if isinstance(x, int)].copy()
    if (-1 in new_aff_ids) & (len(new_aff_ids) > 1):
        return [x for x in new_aff_ids if x != -1].copy()
    elif not new_aff_ids:
        return [-1]
    else:
        return new_aff_ids.copy()

In [0]:
aff_string_table \
    .withColumn('add_affs_1', string_matching_function_udf(F.col('original_affiliation'))) \
    .withColumn('temp_affs', F.array_distinct(F.array_union(F.col('add_affs_1'), F.col('current_affs')))) \
    .withColumn('temp_affs_1', remove_negative_1(F.col('temp_affs'))) \
    .withColumn('affs_removed', matching_based_on_current_affs_udf(F.col('temp_affs_1'), F.col('original_affiliation'))) \
    .withColumn('temp_affs_2', remove_negative_1(F.col('affs_removed'))) \
    .withColumn('row_changed', check_for_change(F.col('current_affs'), F.col('temp_affs_2'))) \
    .filter(F.col('row_changed')==1) \
    .select('original_affiliation', 'current_affs', F.col('temp_affs_2').alias('affiliation_ids_override')) \
    .write.mode('overwrite')\
    .parquet(f"{iteration_save_path}strings_for_v2_table_dist/")

In [0]:
final_counts_data = spark.read.parquet(f"{iteration_save_path}strings_for_v2_table_dist/")
final_counts_data.cache().count()

80227

In [0]:
display(final_counts_data.sample(0.01))

original_affiliation,current_affs,affiliation_ids_override
"Université de Paris, Institut Cochin, CNRS UMR8104, INSERM U1016, F-75014 Paris, France","List(4210141964, 204730241, 154526488)","List(154526488, 204730241, 4210141964, 1294671590)"
"Laboratoire de Chimie et Physique Quantiques, UMR 5626 CNRS—Université Toulouse III–Paul Sabatier 3 , 118 Route de Narbonne, F-31062 Toulouse, France","List(4210144120, 134560555)","List(4210144120, 134560555, 1294671590)"
"Microscopic Imaging Center, Radboud Institute for Molecular Life Sciences, Radboudumc, Nijmegen, The Netherlands","List(4210109357, 2802934949)","List(145872427, 4210109357, 2802934949)"
"State Key Laboratory of Synthetic Chemistry, Department of Chemistry, The University of Hong Kong, Hong Kong, China. huangzx@hku.hk.",List(4210130449),"List(4210130449, 889458895)"
"Center for Advanced Studies and Department of Physics and Astronomy, University of New Mexico, Albuquerque, New Mexico 87131; Optical Sciences Center, University of Arizona, Albuquerque, New Mexico 87131","List(169521973, 4210122332, 138006243)","List(138006243, 169521973)"
"College of Anesthesia, Guizhou Medical University, Guizhou Province, Guiyang 550004, China; Department of Anesthesiology, The Affiliated Hospital of Guizhou Medical University, Guizhou Province, Guiyang 550004, China; Guizhou Medical University Key Laboratory of Anesthesia and Pain Mechanism Research, Guizhou Province, Guiyang, 550004, China. Electronic address: zouxiaohuazxh@gmc.edu.cn.",List(4210109381),"List(149137203, 4210109381)"
"Ecole Normale Supérieure, PSL Research University, Institut de Biologie de l'ENS (IBENS), INSERM, CNRS, Nervous System Development Team, F-75005 Paris, France","List(4210129409, 154526488, 29607241, 2746051580)","List(4210129409, 1294671590, 29607241, 154526488, 2746051580)"
"Jean-Jacques ROSÉ a partagé ses activités entre l'entreprise, l'administration (Ministère de l'Équipement), l'enseignement (École Nationale de Commerce, puis Université Paris IV Sorbonne) et la recherche. Il a accompagné les entreprises confrontées à des mutations toujours plus radicales. Parmi ses publications : avec E. Pisani, in Revue mensuelle Hommes et Citoyens, (1967-1968)",List(null),List(39804081)
"Department of Ophthalmology, Ambroise Paré Hospital, AP-HP, UVSQ, Paris Saclay University, 91190 Gif-sur-Yvette, France",List(null),"List(195731000, 277688954, 4210097159)"
"National Reference Centre for Herpesviruses, Virology Department, Team 3 THERAVIR, and AP-HP, Pitié-Salpêtrière—Charles Foix University Hospital, Institut Pierre Louis d’Epidémiologie et de Santé Publique (iPLESP), INSERM, Sorbonne Université, 75013 Paris, France","List(4210133907, 154526488)","List(154526488, 39804081, 4210133907, 4210097159)"


### Updating override table and putting works in most_things queue

In [0]:
@udf(returnType=StringType())
def process_block_affiliation_change(block):
    conn_part = connect_to_db()

    cur_part = conn_part.cursor()

    for row in block:
        _ = process_row_affiliation_change(conn_part, cur_part, row[0], row[1])

    cur_part.close()
    conn_part.close()

def process_row_affiliation_change(connection, curs_update, aff_string, aff_ids):
    dt = datetime.datetime.now(datetime.timezone.utc)
    val = (dt, aff_string)
    sql_string = f"""UPDATE mid.affiliation_string_v2 SET affiliation_ids_override = jsonb_build_array({aff_ids}), updated = %s WHERE original_affiliation = %s"""

    try:
        curs_update.execute(sql_string, val)
        connection.commit()
    except:
        print("Error while updating row in PostgreSQL:", val)
        connection.rollback()

In [0]:
strings_to_change = spark.read.parquet(f"{iteration_save_path}strings_for_v2_table_dist/")

In [0]:
strings_to_change_rand_int = int(strings_to_change.count()/200)

In [0]:
display(strings_to_change \
    .select('original_affiliation', F.concat_ws(",", F.col('affiliation_ids_override')).alias('affiliation_ids_override')))

original_affiliation,affiliation_ids_override
"#N# 3 Institute of Signal Processing Tampere University of Technology Tampere, Finland",4210133110
") Department of Neonatology, Toho University Omori Medical Center, Tokyo, Japan","129634264,4210095039"
". CNRS-CEMHTI, UPR3079, Université d'Orléans, 1D avenue de la recherche scientifique, 45071 Orléans Cedex, France","4210139596,12449238,1294671590"
". Institute for Climate and Carbon Neutrality, The University of Hong Kong, Hong Kong, China",889458895
".Paris Descartes University, Department of Rheumatology, Hôpital Cochin, AP-HP, EULAR center of excellence, Paris , France","204730241,4210092774,4210097159"
"1Géosciences Rennes, UMR CNRS 6118, OSUR, Université Rennes 1, 35042 Rennes Cedex, France","4210096833,1294671590"
ALISTORE-European Research Institute CNRS FR 3104 Hub de l'Energie 80039 Amiens France,1294671590
"AP-HP, Hôpital Raymond Poincaré, Service de Microbiologie et Hygiène, Garches, France","4210134648,4210097159"
"AP-HP, hôpital Paul Brousse, Laboratoire de virologie, Villejuif 94800, France","4210164847,4210097159"
"APC, AstroParticule et Cosmologie, Université de Paris, CNRS/IN2P3, CEA/IRFU, Observatoire de Paris, 10, rue Alice Domon et Léonie Duquet, 75205, Paris Cedex 13, France","204730241,1294671590,4210165232,2802090066,4210133362,2738703131"


In [0]:
strings_changed = strings_to_change \
    .select('original_affiliation', F.concat_ws(",", F.col('affiliation_ids_override')).alias('affiliation_ids_override')) \
    .select(F.array([F.col('original_affiliation'), F.col('affiliation_ids_override')]).alias('data_to_update')) \
    .withColumn('random_int', (F.rand()*strings_to_change_rand_int+1).cast(IntegerType())) \
    .groupBy('random_int').agg(F.collect_list(F.col('data_to_update')).alias('data_to_update')) \
    .repartition(80) \
    .withColumn('changes_done', process_block_affiliation_change(F.col('data_to_update')))

strings_changed.cache().count()

401

In [0]:
current_date = datetime.datetime.now().strftime("%Y-%m-%d")

In [0]:
test_query = \
    f"""INSERT INTO queue.run_once_work_add_most_things(work_id, rand)
    SELECT DISTINCT paper_id, random()
    FROM mid.affiliatio
    JOIN mid.affiliation_string_v2
    ON mid.affiliation.original_affiliation = mid.affiliation_string_v2.original_affiliation
    WHERE mid.affiliation_string_v2.updated>'{current_date} 00:00' ON CONFLICT DO NOTHING;"""

In [0]:
connection = connect_to_db()
curs_update = connection.cursor()

In [0]:
try:
    curs_update.execute(test_query)
    connection.commit()
except:
    print("Error while updating row in PostgreSQL")
    connection.rollback()

In [0]:
curs_update.execute("SELECT COUNT(*) FROM queue.run_once_work_add_most_things;")
results = curs_update.fetchone() 
print(results)

(1169223,)


In [0]:
curs_update.close()
connection.close()